In [15]:
!pip install transformers==2.8.0
!pip install torch==1.4.0

##DS2 Abstract Summarizer
This solution uses the T5 transformer from the hugging face library as it is convenient and an efficient model to use for the purpose of abstract summarization.

In [3]:
import torch
import json
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from gensim.summarization.summarizer import summarize

In [4]:
paragraphs = pd.read_excel('TASK.xlsx', headers=True) #load the dataset
paragraphs['introduction'] = paragraphs['Unnamed: 1'] #cleanup columns
paragraphs = paragraphs.drop(columns=['TEST DATASET','Unnamed: 1'],axis=1)
paragraphs = paragraphs.drop(paragraphs.index[0])#cleanup rows
new_index = list(range(0,len(paragraphs)))#reorder index
paragraphs['index'] = new_index
paragraphs = paragraphs.set_index('index')
print(len(paragraphs))
paragraphs.head()

1000


,introduction
index,
0,Acnesol Gel is an antibiotic that fights bacte...
1,Ambrodil Syrup is used for treating various re...
2,Augmentin 625 Duo Tablet is a penicillin-type ...
3,Azithral 500 Tablet is an antibiotic used to t...
4,Alkasol Oral Solution is a medicine used in th...


In [5]:
model = T5ForConditionalGeneration.from_pretrained('t5-small') #loading pretrained t5 model
tokenizer = T5Tokenizer.from_pretrained('t5-small') #loading the pretrained t5 tokenizer
device = torch.device('cpu')

In [6]:
def abstract_summarizer(text,len):
  preprocess_text = text.strip().replace("\n","")
  t5_prepared_Text = "summarize: "+preprocess_text
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

  # summmarizing the content using the t5 model
  try:
     summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=len,
                                        max_length=len,
                                        early_stopping=True)
  except:
    text = summarize(text,ratio=0.2, word_count=512, split=False)
    preprocess_text = text.strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=len,
                                        max_length=len,
                                        early_stopping=True)
  
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return output

In [11]:
abstract_summary =[]
sum_len=int(input('What is the desired word count of the summary? Ideal length is around 50 '))
for i in paragraphs.introduction:
  abstract_summary.append(abstract_summarizer(i,sum_len))

paragraphs['abstract'] = abstract_summary
#execute the following code to output the code as an excel file marked as 'output.xlsx'
#paragraphs.to_excel("output.xlsx")
paragraphs.head()

,introduction,abstract
index,,
0,Acnesol Gel is an antibiotic that fights bacte...,Acnesol Gel is only meant for external use and...
1,Ambrodil Syrup is used for treating various re...,Ambrodil Syrup is used for treating various re...
2,Augmentin 625 Duo Tablet is a penicillin-type ...,Augmentin 625 Duo Tablet is a penicillin-type ...
3,Azithral 500 Tablet is an antibiotic used to t...,azithral 500 Tablet is an antibiotic used to t...
4,Alkasol Oral Solution is a medicine used in th...,alkasol Oral Solution can be taken with or wit...
